[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rslab-ntua/MSc_GBDA/blob/master/2022/Lab1b_torch.ipynb)

In [ ]:
# Download data, unzip
!gdown https://drive.google.com/uc?id=1XxBBah4J3wmSAMFq8lBFc06vGWFiy1TZ
!unzip GBDA2020_ML1.zip

In [ ]:
# Read data
DATA_ROOT = "partB/"

CLASS_NAMES = [
    "Alfalfa",
    "Corn-notill",
    "Corn-mintill",
    "Corn",
    "Grass-pasture",
    "Grass-trees",
    "Grass-pasture-mown",
    "Hay-windrowed",
    "Oats",
    "Soybean-notill",
    "Soybean-mintill",
    "Soybean-clean",
    "Wheat",
    "Woods",
    "Buildings-Grass-Trees-Drives",
    "Stone-Steel-Towers"
]

## Handle data. Datasets and DataLoaders

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
from copy import copy

# Build a custom pytorch Dataset-compatible class
class IndianPinesDataset(Dataset):    
    def __init__(self, data_root, transforms=[]):
        super().__init__()
        self.transforms: list = copy(transforms)
        self._build(data_root)
        
    def _build(self, data_root) -> None:
        img = np.load(os.path.join(data_root, "indianpinearray.npy"))
        gt_img = np.load(os.path.join(data_root, "IPgt.npy"))
        
        valid_mask = gt_img > 0

        self.X = img[valid_mask].reshape(-1, 200).astype(np.float32)
        self.y = gt_img[valid_mask].reshape(-1).astype(int) - 1 # "-1" is to compensate for "no_data" class "0" in original_data
    
    def apply_std_scaler(self, indices):
        scaler = StandardScaler()
        scaler.fit(self.X[np.array(indices)])
        self.X = scaler.transform(self.X)
        
    def __getitem__(self, index):
        '''
        Function to retrieve dataset elements (needed)
        '''
        X, y = self.X[index], self.y[index]
        for T in self.transforms:
            X, y = T(X, y)
        return X, y
    
    def __len__(self) -> int:
        '''
        Function to retrieve the total number of samples in dataset (needed)
        '''
        return len(self.X)
        


In [ ]:
# Initialize a dataset instance
dset = IndianPinesDataset(DATA_ROOT)
print("Samples in dataset: ", len(dset))


train_dset, val_dset = random_split(dset, [int(0.7*len(dset)), len(dset)-int(0.7*len(dset))], generator=torch.Generator().manual_seed(2022))

print("Max value for the first sample in 'val' (before scaling): ", val_dset[0][0].max())
dset.apply_std_scaler(train_dset.indices)
print("Max value for the first sample in 'val' (after scaling)\t: ", val_dset[0][0].max())


#  Initialize dataloaders (batching / tensor-casting / shuffling / etc.)
BATCH_SIZE = 64
train_dloader = DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True)
val_dloader = DataLoader(val_dset, batch_size=BATCH_SIZE, shuffle=False)

# Inspect the first batch of samples
for s in train_dloader:
    X, y = s
    print(f"Sample's X type: {type(X)}, dtype: {X.dtype}, shape: {X.size()}")
    print(f"Sample's y type: {type(y)}, dtype: {y.dtype}, shape: {y.size()}")
    break

## Define a MLP model

In [ ]:
from torch import nn

class MLP(nn.Module):
    def __init__(self, in_features: int, num_classes: int):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Forward-pass
        '''
        return self.model(x)
    
model = MLP(200, 16)
print("MLP's output shape: ", model(next(iter(val_dloader))[0]).size())

## Training loop

In [ ]:
from torch.optim import Adam
from torch.nn import functional as F

LEARNING_RATE = 1e-4
NUM_EPOCHS = 100

# Transfer model to GPU
model = model.cuda()

# Define an optimizer
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)


train_losses = []
val_losses = []
val_oa = []

# Training loop
for e in range(NUM_EPOCHS):
    
    # loop over training samples + train for one epoch
    total_loss = 0
    model.train()
    for batch_sample in train_dloader:
        X = batch_sample[0].cuda()
        y = batch_sample[1].cuda()
        
        # Clear gradients
        optimizer.zero_grad()
        
        # Infer with the model
        preds = model(X)
        
        # Compute CCE loss
        # loss = F.cross_entropy(preds, y)
        # Alternatively (often preferred:)
        loss = F.nll_loss(torch.log_softmax(preds, dim=-1), y)
        
        # Back-propagation!
        loss.backward()
        # Step optimizer
        optimizer.step()
        
        total_loss += loss.detach().cpu()
    train_losses.append(float(total_loss)/len(train_dloader))
    
    
    # Validation step
    correct = 0
    total = 0
    total_loss = 0
    
    model.eval()
    for batch_sample in val_dloader:
        X = batch_sample[0].cuda()
        y = batch_sample[1].cuda()
        
        # Infer with the model
        with torch.no_grad():
            preds = model(X)
        # Compute CCE loss
        loss = F.cross_entropy(preds, y, reduction='mean')
        
        total_loss += loss.detach().cpu()
        correct += float((torch.argmax(preds, dim=-1) == y).sum())
        total += len(y)
    
    val_losses.append(float(total_loss)/len(val_dloader))
    val_oa.append(float(correct/total))
    
    if (e+1) % 10 == 0:
        print("Epoch ", e+1)
        print("Total training loss: ", train_losses[-1])
        print("Total validation loss: ", val_losses[-1])
        print("Overall accuracy: ", val_oa[-1])
    

## Standard training plots

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure()
plt.title("Loss v epochs")
plt.plot(range(1, NUM_EPOCHS+1), train_losses, '-r')
plt.plot(range(1, NUM_EPOCHS+1), val_losses, '-g')

plt.figure()
plt.title("Validation Overall Accuracy v epochs")
plt.plot(range(1, NUM_EPOCHS+1), val_oa, '-g')


# Multi-class classification metrics

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
import numpy as np

predictions_list = []
targets_list = []
model.eval()
for batch_sample in val_dloader:
    X = batch_sample[0].cuda()
    y = batch_sample[1].numpy()
    targets_list.append(y)
    
    # Infer with the model
    with torch.no_grad():
        preds = model(X)
    
    predictions_list.append(torch.argmax(preds, dim=-1).cpu().numpy())
predictions = np.concatenate(predictions_list, axis=0)
targets = np.concatenate(targets_list, axis=0)


cM = confusion_matrix(targets, predictions)
# Normalize CM to precision metric
cm_prec = cM / cM.sum(axis=0)
# Normalize CM to recall metric
cm_rec = (cM.T / cM.sum(axis=1)).T


disp = ConfusionMatrixDisplay(confusion_matrix=cM,
                               display_labels=CLASS_NAMES)
plt.figure(figsize=(20,20), dpi=100)
ax = plt.axes()

disp.plot(ax=ax)
plt.xticks(rotation=90)
plt.show()

print(classification_report(targets, predictions, target_names=CLASS_NAMES))